In [77]:
# coding:UTF-8
# importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
% matplotlib inline

import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [78]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.

    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    # defining a blank mask to start with
    mask = np.zeros_like(img)
    # defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    # filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    # returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [79]:
def yellow_transform(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([40, 100, 20])
    upper_yellow = np.array([100, 255, 255])
    mask = cv2.inRange(image_hsv, lower_yellow, upper_yellow)
    # yellowthings = cv2.bitwise_and(image, mask)
    # cv2.imshow('yellowthings',mask)
    # cv2.waitKey(0)
    return mask

In [80]:
def draw_lines(img,lines,color,thickness):
    if lines is not None:
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(img,(x1,y1),(x2,y2),color,thickness)

In [81]:
# 霍夫变换函数
def hough_lines(img):
    """
    `img` should be the output of a Canny transform.

    Returns an image with hough lines drawn.
    """
    rho = 2
    theta = np.pi/180
    threshold = 15
    min_line_len = 60
    max_line_gap = 30
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len,
                            maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, [255,0,0], 2)
    cv2.imshow('hough_image',line_img)
    return lines,line_img

In [82]:
def Lane_finding(image):
    # image = mpimg.imread("/Users/zhangzheng/Desktop/lane_lines_finding/test_images/solidWhiteCurve.jpg")
    # image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_yellow = yellow_transform(image)
    for i in range(540):
        for j in range(960):
            if image_gray[i][j] >= image_yellow[i][j]:
                image_gray[i][j] = image_gray[i][j]
            else:
                image_gray[i][j] = image_yellow[i][j]
    image_blur = cv2.GaussianBlur(image_gray, (5,5), 0)
    low_threshold = 100
    high_threshold = 350
    image_canny = cv2.Canny(image_blur, low_threshold, high_threshold)
    cv2.imshow('image_canny', image_canny)
    # 图像像素行数
    rows = image_canny.shape[0]  # 540行
    # 图像像素列数
    cols = image_canny.shape[1]  # 960列
    left_bottom = [0, rows]  # [0,540]
    right_bottom = [cols, rows]  # [960,540]
    apex = [cols / 2, rows*0.6]  # [480,310]
    vertices = np.array([[left_bottom, right_bottom, apex]], np.int32)
    print(vertices)
    roi_image = region_of_interest(image_canny, vertices)
    cv2.imshow('roi_image', roi_image)
    lines,hough_image = hough_lines(roi_image)
    return hough_image


In [83]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    line_image=Lane_finding(image)
    result=cv2.addWeighted(line_image, 5, image, 0.2, 0)
    return result

In [84]:
challenge_output = '/Users/zhangzheng/Desktop/lane_lines_finding/test_videos_output/challenge_output1.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
# clip1 = VideoFileClip("/Users/zhangzheng/Desktop/lane_lines_finding/test_videos_output/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("/Users/zhangzheng/Desktop/lane_lines_finding/test_videos/challenge.mp4")
print clip1
challenge_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time challenge_clip.write_videofile(challenge_output, audio=False)

<moviepy.video.io.VideoFileClip.VideoFileClip instance at 0x1c2a4f1b90>





t:   0%|          | 0/251 [00:00<?, ?it/s, now=None]

[[[   0  720]
  [1280  720]
  [ 640  432]]]
Moviepy - Building video /Users/zhangzheng/Desktop/lane_lines_finding/test_videos_output/challenge_output1.mp4.
Moviepy - Writing video /Users/zhangzheng/Desktop/lane_lines_finding/test_videos_output/challenge_output1.mp4






t:   1%|          | 2/251 [00:00<01:21,  3.05it/s, now=None]

[[[   0  720]
  [1280  720]
  [ 640  432]]]


...




t: 100%|█████████▉| 250/251 [02:43<00:00,  1.40it/s, now=None]

[[[   0  720]
  [1280  720]
  [ 640  432]]]





t: 100%|██████████| 251/251 [02:44<00:00,  1.41it/s, now=None]

[[[   0  720]
  [1280  720]
  [ 640  432]]]


[[[   0  720]
  [1280  720]
  [ 640  432]]]
Moviepy - Done !
Moviepy - video ready /Users/zhangzheng/Desktop/lane_lines_finding/test_videos_output/challenge_output1.mp4
CPU times: user 2min 39s, sys: 26.9 s, total: 3min 6s
Wall time: 2min 46s
